In [1]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [2]:
# %load_ext autoreload
# %autoreload 2

import sys
import os
from datetime import timedelta
import pandas as pd

# Add src directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store


In [3]:
# Step 1: Setup and Imports
import pandas as pd
import datetime
import requests
import hopsworks
from src.inference import (
    get_feature_store,
    get_model_predictions,
    load_model_from_registry,
    transform_ts_data_info_features
)
import src.config as config

In [4]:
# Step 2: Connect to Hopsworks
print("🔐 Connecting to Hopsworks...")
fs = get_feature_store()

🔐 Connecting to Hopsworks...
2025-05-10 06:52:42,586 INFO: Initializing external client
2025-05-10 06:52:42,586 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 06:52:44,107 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214682


In [5]:
# Step 3: Load Model from Registry
print("📦 Loading latest model...")
model = load_model_from_registry()

📦 Loading latest model...
2025-05-10 06:52:46,736 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 06:52:46,742 INFO: Initializing external client
2025-05-10 06:52:46,743 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 06:52:47,487 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214682


In [6]:
# Step 4: Retrieve Recent Feature Data
print("📡 Fetching recent time-series features...")
fv = fs.get_feature_view(name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION)


📡 Fetching recent time-series features...


In [7]:
# Compute fetch bounds
latest_hour = fv.get_batch_data().pickup_hour.max()
window_size = 24 * 28
fetch_from  = latest_hour - pd.Timedelta(hours=window_size + 1)
fetch_to    = latest_hour

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.91s) 


In [8]:
# Pull feature data
ts = fv.get_batch_data(start_time=fetch_from, end_time=fetch_to)
ts = ts.loc[lambda df: df.pickup_hour.between(fetch_from, fetch_to)].sort_values(["pickup_location_id", "pickup_hour"])


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.66s) 


In [9]:
# Step 5: Prepare Inference Data
features = transform_ts_data_info_features(ts, feature_col="rides", window_size=window_size, step_size=1)
features["target"] = 0  # dummy target for pipeline

In [10]:
# Step 6: Make Prediction
pred_df = get_model_predictions(model, features)
pred_df = pred_df.rename(columns={"predicted_demand": "predicted_rides"})
pred_df["pickup_hour"] = latest_hour + pd.Timedelta(hours=1)
print(pred_df.head())

  pickup_location_id  predicted_rides               pickup_hour
0               5329                6 2024-01-01 00:00:00+00:00
1            5329.03                6 2024-01-01 00:00:00+00:00
2               6140                4 2024-01-01 00:00:00+00:00
3            6140.05                4 2024-01-01 00:00:00+00:00
4               6948               10 2024-01-01 00:00:00+00:00


In [12]:
pred_df.to_csv("latest_prediction.csv", index=False)
print("✅ Prediction saved to latest_prediction.csv")


✅ Prediction saved to latest_prediction.csv
